<a href="https://colab.research.google.com/github/baveja08/BTP/blob/main/best_ligand_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

df = pd.read_excel('Copy of new_data (2)(1).xlsx', sheet_name=1)
df.head(20)

,Unnamed: 0,Metal,Ligand/mof,Potential,Electrolyte,Concentration(M),FE-H2,FE-CO,FE-CH4,FE-HCOOH,FE-C2H4,FE-C2H6,FE-CH3COOH,FE-C2H5OH,FE-C3H7OH,FE-C2+,FE TOTAL,Current density (mA cm-2)
0,NaN,Cu,cyclam,−1.26,KHCO3,0.1,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,NaN
1,NaN,Cu,purpurin,-1.236,KHCO3,0.1,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,NaN
2,NaN,Cu,Cu foil,−1.15,KHCO₃,0.1,0.0,0.0,0.0,20.7,0.0,0.0,0.0,0.0,0.0,0.0,20.7,6.40
3,NaN,Cu,Salen (NO₂ anchored),−1.087,KHCO3,0.1,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,NaN
4,NaN,Cu,Cu-N–C-800,−1.4,KHCO₃,1.0,0.0,0.0,0.0,0.0,24.8,0.0,0.0,0.0,0.0,24.8,24.8,6.84
5,NaN,Cu,macrocyclic,−0.990,KHCO3,0.1,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0,NaN
6,NaN,Cu,Polyacrylamide,-0.96,NaHCO₃,0.1,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,26.0,26.0,NaN
7,NaN,Cu,HKUST-1,-1.16,KHCO₃,0.5,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,4.40
8,NaN,Cu,Hex-ZCu-O,−1.2,KHCO₃,0.1,0.0,0.0,0.0,32.5,0.0,0.0,0.0,0.0,0.0,0.0,32.5,6.50
9,NaN,Cu,Salen (NH₂ anchored),−1.190,KHCO3,0.1,0.0,4.5,5.5,4.5,7.7,0.0,11.1,0.0,0.0,20.3,35.9,NaN


In [49]:
df['FE TOTAL'] = pd.to_numeric(df['FE TOTAL'], errors='coerce')
df['FE-HCOOH'] = pd.to_numeric(df['FE-HCOOH'], errors='coerce')

# Drop rows where either is NaN (i.e., invalid)
df_clean = df.dropna(subset=['FE TOTAL', 'FE-HCOOH'])

# Standardize the features
scaler = StandardScaler()
features = df_clean[['FE TOTAL', 'FE-HCOOH']]
features_scaled = scaler.fit_transform(features)

# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df_clean['Cluster'] = kmeans.fit_predict(features_scaled)

# Identify clusters with high average FE-HCOOH
cluster_means = df_clean.groupby('Cluster')['FE-HCOOH'].mean()
high_hcooh_clusters = cluster_means[cluster_means > cluster_means.median()].index

# Filter only those clusters
filtered_df = df_clean[df_clean['Cluster'].isin(high_hcooh_clusters)]

# Sort and get top 15 by FE-HCOOH
top_15 = filtered_df.sort_values(by='FE-HCOOH', ascending=False).head(15)

# Select final columns (only include if they exist)
required_columns = ['Metal', 'Ligand/mof', 'FE TOTAL', 'FE-HCOOH', 'Potential', 'Electrolyte']
existing_columns = [col for col in required_columns if col in top_15.columns]
top_15_result = top_15[existing_columns].reset_index(drop=True)

# Display result
top_15_result= top_15_result.drop([0, 4])
top_15_result.reset_index(drop=True)



<ipython-input-49-9f2db7741413>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Cluster'] = kmeans.fit_predict(features_scaled)


,Metal,Ligand/mof,FE TOTAL,FE-HCOOH,Potential,Electrolyte
0,Cu,Rubeanate-MOF,98.00,98.00,1.68,KHCO₃
1,Pd,Pb₂(CO₂)₂(OH)₂,95.10,95.10,−1.2,KHCO₃
2,Bi,Bi(BiD),95.00,95.00,−0.97,KHCO₃
3,Cu,Ndf S50/Sus/Cu,90.00,90.00,-0.99,CsHCO₃
4,Cu,Cu-P1,87.00,87.00,-0.47,KOH
5,Cu,AEI-0D-Cu nanosheets,85.10,85.10,-0.8,KOH
6,Cu,CuePfHeAg,83.20,83.20,-1.04,KOH
7,Cu,Cu nanoribbons,82.30,82.30,-1.23,KOH
8,Cu,Cu-CoPC,82.00,82.00,-0.81,KOH
9,Pd,tintmab¹⁰ (NHC),82.00,82.00,-0.57,KHCO₃


In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb

# === Step 1: CLEAN YOUR DATA ===

# 1.1 Replace Unicode minus (−) with normal minus (-) everywhere
df = df.applymap(lambda x: x.replace('−', '-') if isinstance(x, str) else x)

# 1.2 Clean Potential column
def clean_potential(x):
    if isinstance(x, str):
        x = x.split('V')[0].strip()  # Remove "V vs RHE" or similar
    try:
        return float(x)
    except:
        return None

df['Potential'] = df['Potential'].apply(clean_potential)

# 1.3 Clean Concentration(M) column
def clean_concentration(x):
    try:
        return float(str(x).strip())
    except:
        return None

df['Concentration(M)'] = df['Concentration(M)'].apply(clean_concentration)

# 1.4 Convert all FE columns to numeric
fe_cols = ['FE-H2', 'FE-CO', 'FE-CH4', 'FE-HCOOH', 'FE-C2H4', 'FE-C2H6',
           'FE-CH3COOH', 'FE-C2H5OH', 'FE-C3H7OH', 'FE-C2+', 'FE TOTAL']

for col in fe_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 1.5 Drop rows with any missing required values
required_columns = ['Metal', 'Ligand/mof', 'Electrolyte', 'Potential', 'Concentration(M)'] + fe_cols
df = df.dropna(subset=required_columns)

# === Step 2: MODEL SETUP ===

# Columns
categorical_features = ['Metal', 'Ligand/mof', 'Electrolyte']
numeric_features = ['Potential', 'Concentration(M)']  # Only these numeric features

# Inputs and outputs
X = df[categorical_features + numeric_features]
y_total = df['FE TOTAL']
y_hcooh = df['FE-HCOOH']

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ]
)

# Pipelines for Random Forest
model_total_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

model_hcooh_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Pipelines for XGBoost
model_total_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(random_state=42))
])

model_hcooh_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(random_state=42))
])

# === Step 3: FIT THE MODELS ===

# Fit RandomForest models
model_total_rf.fit(X, y_total)
model_hcooh_rf.fit(X, y_hcooh)

# Fit XGBoost models
model_total_xgb.fit(X, y_total)
model_hcooh_xgb.fit(X, y_hcooh)



<ipython-input-50-51dd8aea2631>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('−', '-') if isinstance(x, str) else x)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Metal', 'Ligand/mof',
                                                   'Electrolyte']),
                                                 ('num', StandardScaler(),
                                                  ['Potential',
                                                   'Concentration(M)'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, devi...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [51]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# RandomForest - FE TOTAL
mae_total_rf = mean_absolute_error(y_total, predicted_total_rf)
mse_total_rf = mean_squared_error(y_total, predicted_total_rf)
r2_total_rf = r2_score(y_total, predicted_total_rf)

# RandomForest - FE-HCOOH
mae_hcooh_rf = mean_absolute_error(y_hcooh, predicted_hcooh_rf)
mse_hcooh_rf = mean_squared_error(y_hcooh, predicted_hcooh_rf)
r2_hcooh_rf = r2_score(y_hcooh, predicted_hcooh_rf)

# XGBoost - FE TOTAL
mae_total_xgb = mean_absolute_error(y_total, predicted_total_xgb)
mse_total_xgb = mean_squared_error(y_total, predicted_total_xgb)
r2_total_xgb = r2_score(y_total, predicted_total_xgb)

# XGBoost - FE-HCOOH
mae_hcooh_xgb = mean_absolute_error(y_hcooh, predicted_hcooh_xgb)
mse_hcooh_xgb = mean_squared_error(y_hcooh, predicted_hcooh_xgb)
r2_hcooh_xgb = r2_score(y_hcooh, predicted_hcooh_xgb)

# === Step 6: PRINT RESULTS ===

print("Random Forest - FE TOTAL:")
print(f"MAE: {mae_total_rf:.4f}")
print(f"MSE: {mse_total_rf:.4f}")
print(f"R²: {r2_total_rf:.4f}")

print("\nRandom Forest - FE-HCOOH:")
print(f"MAE: {mae_hcooh_rf:.4f}")
print(f"MSE: {mse_hcooh_rf:.4f}")
print(f"R²: {r2_hcooh_rf:.4f}")

print("\nXGBoost - FE TOTAL:")
print(f"MAE: {mae_total_xgb:.4f}")
print(f"MSE: {mse_total_xgb:.4f}")
print(f"R²: {r2_total_xgb:.4f}")

print("\nXGBoost - FE-HCOOH:")
print(f"MAE: {mae_hcooh_xgb:.4f}")
print(f"MSE: {mse_hcooh_xgb:.4f}")
print(f"R²: {r2_hcooh_xgb:.4f}")


Random Forest - FE TOTAL:
MAE: 3.0927
MSE: 31.3018
R²: 0.9248

Random Forest - FE-HCOOH:
MAE: 5.1101
MSE: 77.1194
R²: 0.8452

XGBoost - FE TOTAL:
MAE: 1.9063
MSE: 5.2966
R²: 0.9873

XGBoost - FE-HCOOH:
MAE: 3.9402
MSE: 23.2110
R²: 0.9534


In [65]:
# These were the expected features used in training:
# categorical_features = ['Metal', 'Ligand/mof', 'Electrolyte']
# numeric_features = ['Potential', 'Concentration(M)']

# Create 4 new combinations
new_data = pd.DataFrame({
    'Metal': ['Cu', 'Ni', 'Bi', 'Bi'],
    'Ligand/mof': ['Cu-MOF-A', 'Ni-Bipyridine', 'Pd-Phen', 'CuePfHeAg'],
    'Electrolyte': ['KHCO₃', 'KOH', 'CsHCO₃', 'KHCO₃'],
    'Potential': [-0.85, -1.20, -0.75, -0.95],
    'Concentration(M)': [0.5, 0.5, 0.5, 0.5]
})

# Predict using the correct model pipelines
rf_predictions_total = model_total_rf.predict(new_data)
rf_predictions_hcooh = model_hcooh_rf.predict(new_data)

xgb_predictions_total = model_total_xgb.predict(new_data)
xgb_predictions_hcooh = model_hcooh_xgb.predict(new_data)

# Combine results
results_df = new_data.copy()
results_df['RF Predicted FE TOTAL'] = rf_predictions_total
results_df['RF Predicted FE-HCOOH'] = rf_predictions_hcooh
results_df['XGB Predicted FE TOTAL'] = xgb_predictions_total
results_df['XGB Predicted FE-HCOOH'] = xgb_predictions_hcooh

# Show final output
results_df.reset_index(drop=True, inplace=True)
results_df


,Metal,Ligand/mof,Electrolyte,Potential,Concentration(M),RF Predicted FE TOTAL,RF Predicted FE-HCOOH,XGB Predicted FE TOTAL,XGB Predicted FE-HCOOH
0,Cu,Cu-MOF-A,KHCO₃,-0.85,0.5,82.7467,6.3067,71.641251,25.632023
1,Ni,Ni-Bipyridine,KOH,-1.20,0.5,85.4240,31.2818,96.401161,24.872824
2,Bi,Pd-Phen,CsHCO₃,-0.75,0.5,92.7340,29.2490,95.680389,21.763754
3,Bi,CuePfHeAg,KHCO₃,-0.95,0.5,90.6930,15.0677,95.308945,27.693331


In [66]:
new_data = pd.DataFrame({
    'Metal': ['Bi', 'Pd', 'Ni', 'Bi', 'Bi'],
    'Ligand/mof': ['Rubeanate-MOF', 'Pb₂(CO₂)₂(OH)₂', 'Ndf S50/Sus/Cu	', 'Ndf S50/Sus/Cu', 'Cu-P1'],
    'Electrolyte': ['KHCO₃', 'KHCO₃', 'KHCO₃', 'CsHCO₃', 'KOH'],
    'Potential': [1.68, -1.20, -0.60, -0.99, -0.47],
    'Concentration(M)': [0.5] * 5  # Assuming 0.5M
})

# Predict using trained pipelines
rf_predictions_total = model_total_rf.predict(new_data)
rf_predictions_hcooh = model_hcooh_rf.predict(new_data)

xgb_predictions_total = model_total_xgb.predict(new_data)
xgb_predictions_hcooh = model_hcooh_xgb.predict(new_data)

# Prepare final results DataFrame
results_df = new_data.copy()
results_df['RF Predicted FE TOTAL'] = rf_predictions_total
results_df['RF Predicted FE-HCOOH'] = rf_predictions_hcooh
results_df['XGB Predicted FE TOTAL'] = xgb_predictions_total
results_df['XGB Predicted FE-HCOOH'] = xgb_predictions_hcooh

# Display result
results_df.reset_index(drop=True, inplace=True)
results_df

,Metal,Ligand/mof,Electrolyte,Potential,Concentration(M),RF Predicted FE TOTAL,RF Predicted FE-HCOOH,XGB Predicted FE TOTAL,XGB Predicted FE-HCOOH
0,Bi,Rubeanate-MOF,KHCO₃,1.68,0.5,90.622,43.8440,96.292656,22.738573
1,Pd,Pb₂(CO₂)₂(OH)₂,KHCO₃,-1.20,0.5,87.298,55.5520,93.487152,90.972069
2,Ni,Ndf S50/Sus/Cu\t,KHCO₃,-0.60,0.5,92.376,11.1790,90.336189,33.118088
3,Bi,Ndf S50/Sus/Cu,CsHCO₃,-0.99,0.5,92.744,58.6900,97.623940,84.445786
4,Bi,Cu-P1,KOH,-0.47,0.5,92.716,39.8579,97.071358,15.130872
